<a href="https://colab.research.google.com/github/ezioauditore-tech/AI/blob/main/Invoice_Extraction_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install fastapi uvicorn nest-asyncio pyngrok transformers paddlepaddle paddleocr


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import nest_asyncio
from fastapi import FastAPI, UploadFile, File
from pyngrok import ngrok
from transformers import pipeline
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image
import os
import numpy as np

nest_asyncio.apply()


In [34]:
app = FastAPI()

model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
answers = []
def extract_text_from_image(image_path):
    ocr = PaddleOCR(use_angle_cls=True, lang="en")
    image = Image.open(image_path)
    image_array = np.array(image)
    result = ocr.ocr(image_array)
    text = ''
    for res in result:
        for line in res:
            text += line[1][0]
    return text

@app.post("/extract-information")
async def extract_information(image: UploadFile = File(...)):
    image_path = f"{image.filename}"
    with open(image_path, "wb") as f:
        f.write(image.file.read())
    text = extract_text_from_image(image_path)


    QA_input4 = {
        'question': 'what is the Name of the shop?',
        'context': text
    }
    answers.append("Store Name: " + nlp(QA_input4)['answer'])

    QA_input1 = {
        'question': 'what is the invoice number in the bill?',
        'context': text
    }
    answers.append("Invoice Number: " + nlp(QA_input1)['answer'])

    QA_input2 = {
        'question': 'what is the date in the bill?',
        'context': text
    }
    answers.append("Date: " + nlp(QA_input2)['answer'])

    QA_input3 = {
        'question': 'what is the final total amount or cash amount in the bill?',
        'context': text
    }
    answers.append("Total Amount: " + nlp(QA_input3)['answer'])

    os.remove(image_path)

    return {"answers": answers}
  
@app.get("/answers", response_model=dict)
async def get_answers():
    return {"answers": answers}


In [35]:
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")


Public URL: https://3f74-34-73-169-98.ngrok-free.app


In [36]:
if __name__ == '__main__':
    uvicorn.run(app, host='0.0.0.0', port=8000)
    

INFO:     Started server process [7327]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     117.255.113.62:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     117.255.113.62:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     117.255.113.62:0 - "GET /extract-information HTTP/1.1" 405 Method Not Allowed
INFO:     117.255.113.62:0 - "GET /extract-information HTTP/1.1" 405 Method Not Allowed
[2023/06/06 14:23:24] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_ps

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [7327]
